### Note to grader

I **really** wasn't 100% sure on what we were supposed to do with this pset -- just test the code or also change it? I did that to an extent by adding a few `assert` statements. What was also unclear was whether we were supposed to simply leave things like checking whether  *any* values in the input array are `nan` unimplemented -- I chose to do so to stay within the performance requirement $O(log(n))$. I have, however, documented this in the `binary_search` function docstring.

## Q1.

Use github to integrate our math library from the lab with Travis CI and Coveralls.
In the cell below, put a link to your github `cs207test` repo so we can track your badges.

https://github.com/j0ma/cs207test

## Q2.

Take the implementation of binary search from a previous lecture/lab. Write unit tests for the algorithm (remember we have doctests in there), stripping the doctests down to those that illustrate the interface for a user. How do these doctests deal with the concerns we had?

Make sure you have good test coverage. You will be integrationg with Travis and Coveralls.

In [15]:
def binary_search(da_array: list, needle, left:int=0, right:int=-1) -> int:
    """
    An algorithm that operates in O(lg(n)) to search for an element
    in an array sorted in ascending order.
    
    Parameters
    ----------
    da_array : list
        a list of "comparable"items sorted in non-descending order
    needle: an item to find in the array; it may or may not
        be in the array
    left: int, optional
        the left index in the array to search from. Default 0
    right: int, optional    
        the right index in the array to search to. Default is -1
        in which case we will use the end of the array `len(da_array) - 1`
        
    Returns
    -------
    index: int
        an integer representing the index of `needle` if found, and -1
        otherwise
        
    Notes
    -----
    PRE: 
        - `da_array` is sorted in non-decreasing order (thus items in
        - `da_array` must be comparable: implement < and ==)
        - `left` must be at least 0
        - `right` must be at least 0, at most len(`da_array`) - 1
    POST: 
        - `da_array` is not changed by this function (immutable)
        - returns `index`=-1 if `needle` is not in `da_array`
        - returns an int `index ` in [0:len(da_array)] if
          `index` is in `da_array`
    INVARIANTS:
        - If `needle` in `da_array`, needle in `da_array[rangemin:rangemax]`
          is a loop invariant in the while loop below.
          
    Examples
    --------
    >>> input = list(range(10))
    >>> binary_search(input, 5) # `da_array` contains needle
    5
    >>> binary_search(input, 4.5) # needle not in `da_array`
    -1
    >>> import numpy as np
    >>> binary_search([1,2,np.inf], np.inf) # np.inf is a valid needle
    2
    >>> binary_search(input, 2, 1,3) # subarray used, contains needle
    2
    >>> binary_search(input, 5, 1,3) # subarray used, doesn't contain needle
    -1
    >>> binary_search(input, 2, 2, 2) # rangemin=rangemax; correct position
    2
    >>> binary_search(input, 5, 2, 2) # rangemin=rangemax; incorrect position
    -1

    NOTE: In the interest of speed, the code does not check 
    that ALL values in the input array are not nan, but simply 
    tests `needle` and the midpoint element. Therefore, there are 
    cases where a nan will not break the code.

    >>> binary_search([1,2, 3, np.nan], 3) # this doesn't break
    2

    >>> from pytest import raises
    >>> binary_search([1,2, np.nan, 3], 3)
    raises(AssertionError)

    """

    # import library for handling np.nan
    import numpy as np

    # check precondition that `needle` is not nan
    assert not np.isnan(needle),\
            "Error: `needle` must be comparable"

    assert left >= 0, '`Left` must be at least 0'

    if left==0:
        rangemin = 0
    else:
        rangemin = left
    if right==-1:
        rangemax=len(da_array) - 1
    else:
        rangemax=right
    while True:
        "needle in da_array => needle in da_array[rangemin:rangemax]"   
        if rangemin > rangemax:
            index = -1
            return index

        #If rangemin and rangemax are both very high we do not want overflow,
        #so get the midpoint like this:
        midpoint = rangemin + (rangemax - rangemin)//2

        # the midpoint element should never be np.nan
        assert not np.isnan(da_array[midpoint]),\
                'Error: elements of array should never be NaN'

        if da_array[midpoint] > needle: #lower part
            rangemax = midpoint - 1
        elif da_array[midpoint] < needle:
            rangemin = midpoint + 1
        else:
            index = midpoint
            return index

Writing binsearch.py


In addition, we should be **systematic** about testing our code. You should at-least jave some tests like this:

1. We should test with wierd data, ie a wierd array: does it have NANs, is it numeric? Does it have 0 elelemts? 1 element? 2?...ie test the boundaries

2. Then think of how the needle relates to the above. Try needles less than or greter than the range in the sorted array, besides needles inbetween (in both cases the needle not being in the array). Try needles at the extremes of the array.

3. test the integration of 1 and 2 to make sure something has not gone wrong there.

Note: you can always compare your answers with linear search implemented in python.

For reference, here are some of our concerns from that lab:

#### What's happened to our issue from before?

- What if the value is not there in the array? 
- What if it is there multiple times? 
- what are we returning (why the -1). Are we consistent in our returning?

We return -1 if the element is not in the array. If it is there multiple times, we will return one of them: it is not defined which. We are consistent by always returning an int, choosing one which cannot be an index.

- What if rangemax is so high as to create overflow: 

We fixed it by using the difference and have documented it in the algorithm

- what types are we supporting? . 

It seems that as long as we have a notion of equals `==`, and a notion of `<` to support sorting we are set. We should document this. (edit: documented)

- what happens if we have a NaN in our array? Infty?
If our preconditions are violated by the user, we can do anything. Doing it nicely might be costly. so we wont.


- what if da_array was not an array?

The user violated the pre-conditions. Anything could happen. We could check for a list
but yhen that would hurt a special class which implemented the python sequence protocol

- What happens if array is not sorted: 

The user violated the pre-conditions. We could return an error, violate post conditions. If we sort it we'd violate the o(lg(n)) notion. (fixing it seems dubious). Can we check if its sorted? This is naively O(n) and breaks our performance specifications. We can create a guard to terminate the array at more than n iterations for the infinite case or let the user have enough rope to hang themselves



**Submit** this to us by creating a repo `cs207binsearch` under your userid with a file `binarysearch.py` and accompanying test file(s). Intergrate with Travis CI and Coveralls. Set up badges on the README of your repo. Write the link to your repo below.

https://github.com/j0ma/cs207binsearch